In [21]:
import os

In [22]:
%pwd

'd:\\kidney_proj'

In [23]:
os.chdir("../")

In [24]:
%pwd

'd:\\'

In [25]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    

In [26]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [27]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [36]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [37]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [38]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [44]:

try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "d:\kidney_proj\kidenv\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Anshika\AppData\Local\Temp\ipykernel_23136\1133135634.py", line 8, in <module>
    raise e
  File "C:\Users\Anshika\AppData\Local\Temp\ipykernel_23136\1133135634.py", line 2, in <module>
    config = ConfigurationManager()
  File "C:\Users\Anshika\AppData\Local\Temp\ipykernel_23136\2866535525.py", line 7, in __init__
    self.config = read_yaml(config_filepath)
  File "d:\kidney_proj\kidenv\lib\site-packages\ensure\main.py", line 872, in __call__
    return_val = self.f(*args, **kwargs)
  File "d:\kidney_proj\kidenv\lib\site-packages\cnnClassifier\utils\common.py", line 37, in read_yaml
    raise e
  File "d:\kidney_proj\kidenv\lib\site-packages\cnnClassifier\utils\common.py", line 30, in read_yaml
    with open(path_to_yaml) as yaml_file:
FileNotFoundError: [Errno 2] No such f